# 자연어 처리를 위한 시퀀스 모델링(3)

## Intro
---
이 장에서는 앞서 설명한 시퀀스 모델링 개념을 ***시퀀스-투-시퀀스 모델링(S2S)*** 으로 확장합니다. S2S 모델은 시퀀스를 입력받아 다른 시퀀스를 출력으로 만듭니다. 입력과 출력 시퀀스의 길이는 다를 수 있으며 작업의 활용 범위는 광범위합니다. 예를 들면 이메일 답장 만들기, 프랑스어 문장을 영어로 번역하기, 기사 요약하기 등이 있습니다.  
또 이 8장에서는 시퀀스 모델의 변종인 양방향 모델을 소개합니다. 어텐션 메커니즘을 소개하고 양방향 모델을 설명하며 그 후 이 장에서 설명한 개념을 담은 신경망 기계 번역 구현 과정을 자세히 다룹니다.  




## 시퀀스-투-시퀀스 모델, 인코더-디코더 모델, 조건부 생성
---
S2S 모델은 ***인코더-디코더 모델*** 이라는 신경망 모델의 일종입니다. 인코더-디코더 모델은 인코더와 디코더 모델로 구성되며 일반적으로 두 모델은 함께 훈련됩니다.  

인코더 모델은 입력을 받아 인코딩 또는 표현을 만듭니다. 일반적으로 출력 결과는 벡터 하나이며 인코더는 입력에서 현재 문제와 관련된 중요한 성질을 감지하고, 디코더는 인코딩된 입력을 받아 원하는 출력을 만드는 데에 사용합니다.  

인코더-디코더 모델은 ***조건부 생성 모델*** 의 일종이라 할 수 있습니다. 조건부 생성에서는 입력 표현 대신 일반적인 조건 문맥 c를 사용해서 디코더가 출력을 만듭니다. 조건 문맥 c가 인코더 모델에서 입력되는 조건부 생성 모델은 인코더-디코더 모델과 같은데 주의해야할 점은 조건 문맥은 구조적인 데이터 소스에서 올 수도 있으니 조건 생성 모델이 곧 모두 인코더-디코더 모델은 아니라는 점입니다.  
일기 예보 생성을 예롤 들어보면 온도, 습도, 풍속, 풍향의 값이 날씨 예측 문장을 생성하는 디코더의 조건이 될 수 있습니다.  

입력과 출력 사이의 매핑을 ***정렬*** 이라고 합니다. 전통적으로 S2S 문제를 다루는 솔루션은 대부분 경험으로 얻은 통계적 방법과 엔지니어링을 시도했습니다. 여기서는 다루지 않았지만 Koehn의 논문과 Statmt.org 자료를 추천합니다.  

6장에서 시퀀스 모델이 임의의 길이의 시퀀스를 어떻게 벡터로 인코딩하는지 배우고, 7장에서는 하나의 벡터가 다른 성씨를 생성하도록 신경망을 어떻게 편향시키는지 보았습니다. S2S 모델은 이런 개념을 확장한 것입니다.

## 강력한 시퀀스 모델링: 양방향 순환 모델
---
순환 모델은 시퀀스르르 벡터로 인코딩하는 블랙 박스로 생각할 수 있습니다. 시퀀스를 모델링할 때 지난 단어와 함께 앞으로 나타날 단어를 관찰하면 도움이 됩니다. 다음 문장을 살펴볼까요?  

# **The man who hunts ducks ous on the weekends.** 



이 문장을 왼쪽에서 오른쪽으로만 처리하는 모델과 오른쪽에서 왼쪽으로도 처리하는 모델은 'ducks'를 다르게 표현할 것입니다. 사람은 항상 이런 사후 의미 업데이트를 수행합니다.  

물론 과거와 미래의 정보를 합치면 문장에 있는 단어의 의미를 안정저긍로 표현할 수 있습니다. 이것이 양방향 순환 모델의 목적입니다. 엘만 RNN, LSTM, GRU 등 모든 순환 신경망 모델은 양방향으로 구성할 수 있습니다. 이전에 본 단방향 모델처럼 양방향 모델도 분류나 입력 단어마다 하나의 레이블을 예측하는 시퀀스 레이블링 작업에 사용할 수 있습니다.


## 강력한 시퀀스 모델링: 어텐션
---
위에서 소개해 드린 S2S 모델에는 전체 입력 문장을 하나의 벡터에 밀어 넣는다는(인코딩한다는) 문제가 있습니다. 이런 모델은 매우 짧은 문장에서만 통할 뿐 긴 문장에서는 전체 입력 정보를 감지하지 못한다는 한계점이 있습니다. 왜냐하면 인코딩에 최종 은닉 상태만 사용하는 제약 때문에 그렇습니다. 또한 긴 문장은 시간을 거슬러 역전파할 때 그레이디언트가 소실되어 훈련이 어렵다는 문제도 있습니다.  

일반적으로 사람은 문장의 의미를 추출한 뒤 이를 바탕으로 번역하지 않습니다. 다르게 말하면 사람은 출력을 생성할 때 관련된 입력 부분에 초점을 맞춥니다. 이런 현상을 ***어텐션*** 이라 부릅니다.  

여러분은 읽고 있는 모든 단어에 어텐션을 부여합니다. 기억력이 뛰어나더라도 책 전체를 하나의 문장처럼 읽지 않을 것입니다. 그 대신 한 단어를 읽을 때 주변 단어에 주의를 기울이고 절과 장의 주제에 주의를 기웁입니다.  

이와 비슷하게 전체 입력의 최종 요약이 아니라 입력의 여러 부분에 어텐션을 부여하는 시퀀스 생성 모델을 만들겠습니다. 이를 ***어텐션 메커니즘*** 이라고 부릅니다.  

NLP에 어텐션의 개념을 사용한 첫 번째 모델은 바다나우 등의 2015년 논문에 있는 기계 번역 모델입니다. 그 이후 여러 종류의 어텐션 메커니즘과 어텐션을 향상하는 여러 방법이 개발되었습니다. 이 절에서는 기본적인 어텐션 메커니즘과 어텐션 관련 용어를 몇 가지 소개합니다. 어텐션은 입출력이 복잡한 딥러닝 모델의 성능을 높이는 데에 매우 유용합니다. 사실 바다나우 등의 2015년 논문에서는 어텐션 메커니즘이 없을 때 입력이 길어질수로 BLEU 점수로 측정된 기계 번역 시스템의 성능이 저하됨을 보였습니다. 어텐션을 추가하면 이런 문제가 해결됩니다.  



### 심층 신경망의 어텐션
---


S2S 모델을 생각해 볼까요? 전형적인 S2S 모델에서 각 타임 스텝은 은닉 상태를 생성합니다. 이는 인코더의 해당 타임 스텝에 대응됩니다. 어텐션을 사용할 때는 인코더의 최종 은닉 상태뿐만 아니라 중간 타임 스텝의 은닉 상태도 고려합니다. 혼란스러울 수 있는 용어이지만 인코더의 이런 은닉 상태를 ***값*** (또는 상황에 따라 ***키***)이라고 부릅니다. 어텐션은 디코더의 이전 은닉 상태도 고려하며 이를 ***쿼리*** 라고 부릅니다. 또한 어텐션은 주의를 기울이려는 값의 개수와 차원이 같은 벡터 하나로 표현이 되는데 이를 ***어텐션 벡터*** 또는 ***어텐션 가중치***, 때로는 ***정렬*** 이라 부릅니다. 어텐션 가중치는 인코더 상태(값)와 연결되어 ***문맥 벡터*** 를 생성합니다. 이따금 이를 ***글림스*** 라고도 부릅니다. 전체 문장의 인코딩 대신에 문맥 벡터가 디코더의 입력이 됩니다. 다음 타임 스텝의 어텐션 벡터는 ***호환성 함수*** 를 사용해 업데이트됩니다.  

어텐션을 구현하는 방법은 여러 가지가 있지만, 그 중 가장 간단한 ***콘텐츠 인식 어텐션*** 을 가장 많이 사용합니다. 물론 쿼리 벡터와 키만을 사용하는 ***위치 인식 어텐션*** 도 인기있는 메커니즘입니다. 어텐션 가중치는 일반적으로 0과 1사이의 실수인데 이를 ***소프트 어텐션*** 이라고 합니다. 하지만 어텐션이 0 아니면 1인 이진 벡터를 학습할 수도 있습니다. 이는 ***하드 어텐션*** 이라고 합니다. 입력의 모든 타임 스텝에 대해 인코더의 상태를 사용하는 것을 ***전역 어텐션*** 이라고도 합니다. 반면에 ***지역 어텐션*** 은 현재 타임 스텝 주위에 있는 입력에만 의존합니다.  

기계 번역에서는 명시적으로 정렬 정보를 훈련 데이터로 제공하기도 합니다. 이런 상황에서는 동시에 훈련되는 별도의 신경망을 사용해서 어텐션 함수를 학습하는 ***지도 어텐션*** 메커니즘을 구성할 수 있습니다.  

2017년 논문에서는 트랜스포머 네트워크를 위한 ***멀티헤드 어텐션*** 을 소개합니다. 여러 어텐션 벡터를 사용해서 입력의 다양한 영역을 추적합니다. 또한 ***셀프 어텐션*** 개념을 대중화했습니다. 셀프 어텐션은 입력의 어떤 영역이 다른 영역에 영향을 미치는지 학습하는 메커니즘입니다.  

이미지와 음성처럼 입력의 형태가 다양할 때는 ***멀티모달 어텐션*** 메커니즘을 구성할 수 있습니다. 새롭지만 어텐션에 대한 논문은 이미 방대해서 이 주제가 얼마나 중요하지 알려줍니다.

## 시퀀스 생성 모델 평가
---

Precison, Recall, F1 같은 지표는 정답이 여럿인 모델에는 도움이 되지 않습니다. 예를 들어 프랑스어 문장 하나에서 다양한 영어 번역문이 나올 수 있습니다. 시퀀스 모델은 ***참조 출력*** 이라는 기대 출력으로 평가합니다. 여러 모델을 비교할 때 모델의 출력이 얼마나 참조 출력에 가까운지를 점수로 매깁니다.  

시퀀스 생성 모델에는 평가 방법이 두 종류가 있습니다. 바로 사람 평가와 자동 평가입니다.  
기계 번역에서 사람 평가는 한 명 이상의 사람이 모델 출력에 '좋음' 또는 '나쁨'을 표시하거나 번역을 고치는 방법입니다. 이는 사람이 작업할 때와 유사하게 시스템 출력의 최종 목표와 매우 비슷한 간단한 '에러율'을 만듭니다. 사람 평가가 중요하지만, 평가 속도가 느리고 비용이 많이 들어 자주 사용하지는 않으며 사람들 간의 평가가 다를 수도 있다는 점을 유념해야 합니다. 따라서 사람 평가는 반드시 ***평가자 간의 일치율*** 과 함께 사용해야 합니다. ***HTER*** 는 널리 사용하는 사람 평가 지표입니다. 사람이 번역 결과를 더 자연스럽고 의미에 맞게 개선하려고 추가, 삭제, 이동한 횟수를 헤아려 계산한 가중치가 적용된 편집 거리입니다.  

반면 자동 평가는 실행하기 쉽고 빠릅니다. 생성된 시퀀스를 자동으로 평가하는 지표에는 두 종류가 있습니다. ***n-gram 중복 기반 지표*** 와 ***혼란도(perplexity)***입니다. 이 지표는 여기서 예로 든 기계 번역 외에도 시퀀스 생성과 관련된 다른 작업에도 적용할 수 있습니다. n-gram 중복 기반 지표는 참조와 출력이 얼마나 가까운지 n-gram 중복 통계로 점수를 계산하는데 이때 BLEU, ROUGE, METEOR가 n-gram 중복 기반 지표입니다. 참고로 이 중에서 BLEU는 기계 번역 논문에서 측정 지표로 오랫동안 사용되었습니다.  

점수를 실용적으로 계산하려고 NLTK와 SacreBLEU 패캐지를 사용합니다. 참조 데이터가 있다면 BLEU를 빠르고 쉽게 계산할 수 있습니다.  

혼란도(perplexity) 역시 정보 이론에 기반한 자동 평가 지표입니다. 출력 시퀀스의 확률을 측정할 수 있다면 적용 가능합니다. 

따로 떼어 놓은 데이터셋에서 모델의 혼란도를 측정해 여러 시퀀스 생성 모델을 쉽게 비교할 수 있습니다. 혼란도는 계산하기 쉽지만, 시퀀스 생성 모델의 평가에 사용할 때 여러 문제점이 있습니다.  
첫째로 혼란도는 과장된 지표입니다. 혼란도 공식에는 지수 함수가 있는데 결국 모델 성능(가능도)의 작은 차이가 혼란도에서 큰 차이를 만듭니다. 크게 향상되었다고 착각할 수 있습니다.  
둘째로 다른 지표와 달리 혼란도의 변화는 모델의 오차율에 직접 반영되지 않습니다.  
마지막으로 셋째는 (BLEU 등 n-gram 기반 지표처럼) 혼란도가 향상되더라도 사람이 판단하기에는 향상되었다고 느끼지 못할 수 있습니다.